Парсер TripAdvisor. Часть первая, собирает ссылки на объекты.

Пока количество страниц в полуручном режиме.


Последнее обновление: 10 апреля 2022

## Библиотеки и настройки

In [ ]:
#Загружаю нужные библиотеки

import pandas as pd
import math

import datetime

# регулярки
import re


# для юзер-агента
from fake_useragent import UserAgent

# прогресс-бар
from tqdm.notebook import trange, tqdm

# для пауз при парсинге
from time import sleep

# селениум
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
# настройки юзер-агента

ua = UserAgent()

# юзер-агент для Хрома
user_agent = ua.chrome

In [ ]:
# тестовая страница для Селениума

test_url = 'https://intoli.com/blog/not-possible-to-block-chrome-headless/chrome-headless-test.html'

In [ ]:
# путь к файлам

file_path = 'C:/00_Data/' + input()
proxy_list_path = 'C:/work/keys/proxy_list.txt' # путь и названия со списком прокси

In [ ]:
# загружаю список прокси
# список простой текстовый файл, который содержит прокси в формате: адрес:порт логин:пароль

proxy_list = []

with open(proxy_list_path, 'r') as f:
    for line in f:
        proxy_list += [line.split()]
        
print('Рабочих прокси — ' + str(len(proxy_list)))

In [ ]:
# номер прокси

print('Номер прокси, от 0 до ' + str(len(proxy_list) - 1))
proxy_number = int(input())

In [ ]:
# настройки веб-драйвера, чтобы скрыть автоматическое управление

opt = webdriver.ChromeOptions()
opt.add_argument(f'user-agent={user_agent}')
opt.add_argument('--disable-blink-features')
opt.add_argument('--disable-blink-features=AutomationControlled')
#opt.add_argument(f'--proxy-server={proxy_list[proxy_number][0]}') # "девственный" прокси(:
opt.add_experimental_option("excludeSwitches", ["enable-automation"]) # отключает строку про удаленное управление
opt.add_experimental_option('useAutomationExtension', False)

In [ ]:
print('Login: ' + proxy_list[proxy_number][1])
print('Pass: ' + proxy_list[proxy_number][2])

In [ ]:
# запускаю драйвер, проверяю настройки

#s=Service('C:\Users\boris\anaconda3\chromedriver.exe')
#driver = webdriver.Chrome(service=s, options=opt)

driver = webdriver.Chrome(options=opt)
driver.get(test_url)
driver.maximize_window() # раскрываю окно на максимум

### Собираю урлы

In [ ]:
# ссылка на страницу TripAdvisor

original_url = input()

In [ ]:
# количество страниц

(driver.find_element(By.TAG_NAME, 'section[data-automation="WebPresentation_PaginationLinksList"]')
 .text.split('\n')
)[-1]

In [ ]:
url_part_1 = re.findall(r'.+Activities-c\w\w-', original_url)[0]

In [ ]:
url_part_2 = re.findall(r'(?<=Activities-c\w\w).+', original_url)[0]

In [ ]:
page_list = ['oa00']


for i in range(1, 3):
    page_list.append('oa' + str(i * 30))

In [ ]:
data_urls = []

for page in tqdm(page_list):
    
    driver.get(url_part_1 + page + url_part_2)
    sleep(3)
    
    card_blocks = driver.find_elements(By.TAG_NAME, 'div[data-automation="cardWrapper"]')
    
    for one_card in card_blocks:
        title = one_card.find_element(By.TAG_NAME, 'span[name="title"]').text
        object_url = one_card.find_element(By.TAG_NAME, 'a').get_attribute('href') # первая ссылка из кода
        
        try:
            score = (one_card.find_element(By.TAG_NAME, 'header')
                     .find_element(By.TAG_NAME, 'svg').get_attribute('aria-label'))
            
        except:
            score = 'none'
            
        try:
            score_number = (one_card.find_element(By.TAG_NAME, 'header')
                            .find_element(By.TAG_NAME, 'span[class="WlYyy diXIH bGusc bQCoY"]').text)
            
        except:
            score_number = 0
            
        
        data_urls.append([title, object_url, score, score_number])

In [ ]:
head = ['title', 'object_url', 'score', 'score_number']

work_urls = pd.DataFrame(data_urls, columns=head)

In [ ]:
work_urls.shape

In [ ]:
work_urls.tail(5)